In [ ]:
import sys
from pathlib import Path

AVES_ROOT = Path("..") / ".." / ".."

EOD_PATH = AVES_ROOT / "data" / "external" / "EOD_STGO"
EOD_PATH


In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns

# esto configura la calidad de la imagen. dependerá de tu resolución. el valor por omisión es 80
mpl.rcParams["figure.dpi"] = 150
# esto depende de las fuentes que tengas instaladas en el sistema.
mpl.rcParams["font.family"] = "Fira Sans Extra Condensed"


In [ ]:
zones = gpd.read_file(AVES_ROOT / "data" / "processed" / "scl_zonas_urbanas.json")
zones.plot()


In [ ]:
from aves.data import eod

hogares = eod.read_homes(EOD_PATH)
personas = eod.read_people(EOD_PATH).merge(hogares)

viajes = eod.read_trips(EOD_PATH).merge(personas).merge(hogares)

viajes["PesoLaboral"] = viajes["FactorLaboralNormal"] * viajes["Factor_LaboralNormal"]

viajes = viajes[pd.notnull(viajes["PesoLaboral"])]

viajes.columns


In [ ]:
zonas_diff = (
    viajes[
        (viajes["Proposito"].isin(("Al trabajo", "Al estudio")))
        & (viajes["ModoDifusion"].str.contains("Bip"))
    ]
    .groupby(["Zona"])["PesoLaboral"]
    .sum()
    .rename("uso_tp")
    .to_frame()
)


zonas_diff


In [ ]:
peso_total = (
    viajes[viajes["Proposito"].isin(("Al trabajo", "Al estudio"))]
    .groupby("Zona")["PesoLaboral"]
    .sum()
    .rename("n_viajes")
)

peso_total


In [ ]:
zones_with_values = (
    zones.join(zonas_diff, on="ID", how="inner")
    .drop_duplicates(subset="ID")
    .join(peso_total, on="ID", how="left")
    .assign(proporcion_tp=lambda x: x["uso_tp"] / x["n_viajes"])
)
zones_with_values


In [ ]:
zones_with_values.plot(column="proporcion_tp", cmap="Reds", legend=True)


In [ ]:
from libpysal.weights import Rook, Queen, Kernel, spatial_lag

# w = Queen.from_dataframe(zones_with_values)
w = Kernel.from_dataframe(
    zones_with_values.to_crs("epsg:5361"),
    k=4,
    function="gaussian",
    fixed=False,
    diagonal=False,
)
w.n


In [ ]:
w.bandwidth


In [ ]:
np.diag(w.full()[0])


In [ ]:
sns.heatmap(w.full()[0])


In [ ]:
w.transform = "R"


In [ ]:
sns.heatmap(w.full()[0])


Definición de Spatial Lag

Valor del área de interés: $y_i$

Valor de sus vecinos:

$y_{li} = \sum_j w_{ij} y_j$

In [ ]:
zones_with_values["proporcion_tp_lag"] = spatial_lag.lag_spatial(
    w, zones_with_values["proporcion_tp"]
)
zones_with_values[["proporcion_tp_lag", "proporcion_tp"]].sample(15)


In [ ]:
zones_with_values.plot("proporcion_tp_lag", cmap="Reds", legend=True)


In [ ]:
zones_with_values["proporcion_tp_std"] = (
    zones_with_values["proporcion_tp"] - zones_with_values["proporcion_tp"].mean()
)
zones_with_values["proporcion_tp_lag_std"] = spatial_lag.lag_spatial(
    w, zones_with_values["proporcion_tp_std"]
)


In [ ]:
zones_with_values.plot("proporcion_tp_lag_std", cmap="Reds", legend=True)


In [ ]:
ax = zones_with_values.plot(
    x="proporcion_tp_std", y="proporcion_tp_lag_std", kind="scatter"
)
ax.axvline(0, c="k", alpha=0.5)
ax.axhline(0, c="k", alpha=0.5)


In [ ]:
sns.regplot(data=zones_with_values, x="proporcion_tp_std", y="proporcion_tp_lag_std")


In [ ]:
from scipy.stats import linregress

reg_result = linregress(
    zones_with_values["proporcion_tp_std"], zones_with_values["proporcion_tp_lag_std"]
)
# slope, intercept, rvalue, pvalue
reg_result[0], reg_result[1], reg_result[2], reg_result[3]


Moran's I

$I = \frac{n}{\sum_i \sum_j w_{ij}} \frac{\sum_i \sum_j w_{ij} z_i z_j}{\sum_i z_i^2}$

Donde:

* $n$ es la cantidad de muestras
* $w_{ij}$ es el valor de spatial lag de area $j$ para area $i$
* $z_i$ es $x_i - \bar{x_i}$

In [ ]:
zones_with_values["proporcion_tp"].sample(frac=1.0)


In [ ]:
from esda.moran import Moran

moran = Moran(zones_with_values["proporcion_tp"], w)


In [ ]:
moran.I, moran.p_sim


Local Moran's I

$I_i = \frac{n * z_i}{\sum_i z_i^2} \sum_j w_{ij} z_j$

De hecho:

$I = \sum_{i = 1}^n \frac{I_i}{n}$

In [ ]:
from esda.moran import Moran_Local

lisa = Moran_Local(zones_with_values["proporcion_tp"], w)


In [ ]:
ax = sns.kdeplot(lisa.Is)
sns.rugplot(lisa.Is, ax=ax)


In [ ]:
from splot import esda as esdaplot

esdaplot.lisa_cluster(lisa, zones_with_values, p=1)


In [ ]:
esdaplot.lisa_cluster(lisa, zones_with_values)


In [ ]:
zones_with_values["lisa_group"] = lisa.q * (lisa.p_sim < 0.05).astype(int)
zones_with_values["lisa_group"].value_counts()


In [ ]:
zones_with_values["label"] = zones_with_values["lisa_group"].map(
    {
        0: "Non-Significant",
        1: "HH (Hot Spots)",
        2: "LH",
        3: "LL (Cold Spots)",
        4: "HL",
    }
)
zones_with_values["label"].value_counts()
